# Linear Regression

In this notebook I will implement a linear regression algorithm from scratch using a numerical method called Gradient Descent. A linear regression is a mathematical model that is obtained as the "linear" model of best fit for a given dataset which is then used to predict values outside the dataset.

## What is meant for linear model?
When I think about the use of the term "linear" in mathematics two ideas come to my mind, the first one is a function of the form $f(x)=mx+b$ as its graph in the $x,f(x)$ plane is a line. The other idea that comes to mind is the one used in linear algebra, that of a linear map. A linear map is a function that assigns elements from a vector space $V$ to a vector space $W$ $\left(f:V \rightarrow W\right)$ such that $f(a\boldsymbol{v})=af(\boldsymbol{v})$ and $f(\boldsymbol{v}+\boldsymbol{u})=f(\boldsymbol{v})+f(\boldsymbol{u})$.

Since a linear function $f(x)=mx+b$ is generally not a linear map (in the linear algebra sense) as
$$f(x)+f(y) = mx+b + my+b = m(x+y)+2b = (m(x+y)+b) + b = f(x+y)+b \neq f(x+y)$$
I consider relevant to define what is meant by linearity in this context.

Given a set of numbers $\{x_{0}, x_{1}, ..., x_{n}\}$, which we shall store in a vector $\boldsymbol{x}$, a linear model will be a function that maps $\boldsymbol{x}$ to a scalar, such that 
$$f(\boldsymbol{x})=\sum_{j=0}^{n}\theta_{j}x_{j},$$
where each $\theta_{j}$ is a scalar called parameter. This is an extension of the linear function to $n$ variables, although the particular example used to test the modules is a simple linear function of 1 variable, the modules should also work for any number of parameters.

To define the linear model we may also decide to store the values of the parameters in a vector $\boldsymbol{\theta}$ such that the previous definition is equivalent to the "usual" dot product, which is computed by multiplying corresponding components together and adding the results
$$f(\boldsymbol{x}) = \langle \boldsymbol{x}, \boldsymbol{\theta} \rangle.$$

In machine learning, each of the numbers $x_{j}$ is called a feature, and we use a finite set of vectors containing these features $\boldsymbol{x}^{(i)}$ to deduce the parameters $\boldsymbol{\theta}$ that make the linear model $\langle \boldsymbol{x}, \boldsymbol{\theta} \rangle$ fit the best to the dataset given. We can store all of the vectors $\boldsymbol{x}^{(i)}$ in a matrix $X$, such that each row corresponds to a partiular vector and each column corresponds to a feature. That is $X_{ij}=x^{(i)}_{j}$.

Using the matrix $X$ we can store the result of $f\left(\boldsymbol{x}^{(i)}\right)$ for each $\boldsymbol{x}^{(i)}$ in a vector $\boldsymbol f$ using matrix multiplication:
$$f_{i}=\sum_{j=0}^{n}X_{ij}\theta_{j} \implies \boldsymbol{f}=X\boldsymbol{\theta}.$$

## Best fit?
How can we determine which model has the "best fit"? Well, we can define a function $J$ called cost, such that $J(\boldsymbol{\theta})$ is a measure of how bad a set of parameters is and find $\boldsymbol{\theta}$ such that is cost is minimized. In this notebook we will use half of the mean squared error, which is calculated by halving the average of the squared differences between the actual data points and the model: $$J(\boldsymbol{\theta})=\frac{1}{2m}\sum_{i=1}^{m}\left(\langle \boldsymbol{x}^{(i)}, \boldsymbol{\theta} \rangle - y^{(i)}\right)^{2},$$
such that $m$ corresponds to the number of training examples. The minimum of this function will be the same as that of the mean squared error (the average of the differences between the actual data poitns and the model) but it makes gradient descent easier to compute.

$J$ can also be written using the matrix $X$ as:
$$J\left(\boldsymbol{\theta}\right)=\frac{\langle X\boldsymbol{\theta} - \boldsymbol{y}, X\boldsymbol{\theta} - \boldsymbol{y}\rangle}{2m}=\frac{\left(X\boldsymbol{\theta} - \boldsymbol{y}\right)^{T}\left(X\boldsymbol{\theta} - \boldsymbol{y}\right)}{2m}.$$
The right-most expression is how the cost function will be implemented in this notebook using `numpy`.

## How do we find the minimum?
Funny enough the optimal parameters can be found analitically, that is we can perform a series of operations to calculate the exact values of $\theta_{j}$ that minimize $J$; however in this notebook we will use a numerical method, that is we will find an approximate solution using an algorithm, the algorithm used will be gradient descent. This method comes from an observation in mathematics: for a multivariable function $f(\boldsymbol{x})$ the gradient $\nabla f=\partial_{x_{i}}f$ will always have the direction of steepest ascent, thus by "moving" in the opposite direction $-\nabla f$, we move in the direction of steepest descent. This can be summarized by the equation:
$$\boldsymbol{a}_{n+1}=\boldsymbol{a}_{n}-\alpha\nabla f$$
such that $\alpha$ is a parameter for how big of a "step" should we take in the opposite direction of the gradient. However, since the gradient changes along the surface, the direction of steepest descent changes as well. Thus, the step size must be small to ensure the direction of the gradient is approximately equal in that region and therefore $\alpha$ must be a "small" value (but big enough to converge quickly). Moreover, as $\boldsymbol{a}$ approaches the minimum, the partial derivatives approach 0 and therefore the steps become smaller and smaller which we may recognize as convergence.

In [1]:
#Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget

## Modules

In [2]:
def plotData(x, y, title, xlabel, ylabel):
    fig, ax = plt.subplots()
    ax.scatter(x, y, c="salmon", marker="x", label="Training data")
    ax.set(title = title,
           xlabel = xlabel,
           ylabel = ylabel)
    plt.show()
    return ax

In [3]:
def Cost(X, y, theta):
    m = y.size
    return np.transpose(X @ theta - y) @ (X @ theta - y) / (2*m)

In [4]:
def GradientDescent(X, y, theta, alpha, num):
    """
    This is a function that performs gradient descent to minimize the average of the square residuals.
    --------------------------------------------------------------------------------
    X is a matrix containing the training examples with each column being a feature.
    y is a vector containing the target values for each row of X.
    theta is a vector containing the regression parameters, such that h=Xθ.
    alpha is the learning rate such that Gradient Descent follows θ := θ - α ∂C/∂θ.
    num is the number of iterations desired to run gradient descent.
    """
    J = np.zeros((num,1))
    
    for i in range(num):
        theta = theta - alpha/m * np.transpose(X) @ (X @ theta - y)
        J[i] = Cost(X, y, theta)
        
    return theta, J

## Data

In [5]:
df = pd.read_csv("ex1data1.txt", sep=",")

# Features
X = df.drop("Target", axis=1)

# Targets
y = df["Target"]

# Sample
m = y.size

In [6]:
ax = plotData(X, y,"Population vs Profit","Profit in $10,000s", "Population of City in 10,000s")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
X = np.c_[np.ones(m), X]
theta = np.zeros(2)

## Gradient Descent

In [8]:
print(f"With theta = [0 ; 0] \nCost computed = {Cost(X, y, theta):.2f}")

With theta = [0 ; 0] 
Cost computed = 32.07


In [9]:
print(f"With theta = [-1 ; 2] \nCost computed = {Cost(X, y, np.array([-1, 2])):.2f}")

With theta = [-1 ; 2] 
Cost computed = 54.24


In [10]:
theta, J = GradientDescent(X, y, theta, 0.01, 1500)
theta

array([-3.63029144,  1.16636235])

In [11]:
ax.plot(X[:,1], X@theta, label="Linear regression")
ax.legend(fontsize='large');

## Cost function visualization

In [12]:
theta0_vals = np.linspace(-10, 10, 100)
theta1_vals = np.linspace(-1, 4, 100)

J_vals = np.zeros((theta0_vals.size, theta1_vals.size))

for i in range(theta0_vals.size):
    for j in range(theta1_vals.size):
        t = np.r_[theta0_vals[i], theta1_vals[j]]
        J_vals[i, j] = Cost(X, y, t)     

Theta0, Theta1 = np.meshgrid(theta0_vals, theta1_vals)

J_vals = np.transpose(J_vals)

plt.rcParams['text.usetex'] = True

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(Theta0, Theta1, J_vals, cmap='viridis')
ax.set(title=r'$J(\mathbf{\theta})$',
       xlabel=r'$\theta_{0}$',
       ylabel=r'$\theta_{1}$')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fig = plt.figure()
ax = fig.add_subplot()
ax.contour(Theta0, Theta1, J_vals, np.logspace(-2, 3, 20))
ax.set(title="Contour plot",
       xlabel = r'$\theta_{0}$',
       ylabel = r'$\theta_{1}$')
ax.scatter(theta[0], theta[1], marker="x", label='Theta found by Gradient Descent')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …